# **Procesamiento de Lenguaje Natural**

## **Tarea 2 - Notebook 1**

## Integrantes

* ### Daniel Osorio Cárdenas
* ### Juan Diego Calixto Núñez

Este Notebook incluye los literales I, II, III, IV de la tarea. Estos corresponden a la construcción de los modelos de lenguaje que se usarán en el Notebook 2.

## **I. Creación de los archivos consolidados**

In [1]:
# Configurar los directorios donde se encuentran los archivos de datos
NEWS_FOLDER = '20news-18828'
BLOGS_FOLDER = 'blogs'

Empezamos cargando los archivos de 20news-18828.

In [2]:
import os

# Listar los folders de 20news-18828
news_folder = os.listdir(NEWS_FOLDER)

# Recorremos cada archivo de news
news_texts = []
for folder in news_folder:
    for file_name in os.listdir(NEWS_FOLDER + '/' + folder):
        extracted_text = ''
        with open(NEWS_FOLDER + '/' + folder + '/' + file_name, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                # Agregamos solo las lineas del contenido del archivo
                if not line.startswith('From:') and not line.startswith('Subject:') and not line.startswith('Fax:') and not line.startswith('Phone:') and not line.startswith('Email:') and not line.startswith('INTERNET:'):
                    extracted_text += line + " "
            news_texts.append(extracted_text + "<NEW_DOCUMENT>")

# Guardamos todos los textos en un unico .txt
with open('20N_consolidated.txt', 'w', encoding='utf-8') as file:
    for text in news_texts:
        file.write(text)

# Revisamos el tamaño del archivo .txt
print('Tamaño del archivo 20N_consolidated.txt: ', os.path.getsize('20N_consolidated.txt'), 'bytes = ', round(os.path.getsize('20N_consolidated.txt') / 1048576, 2), 'MB')

Tamaño del archivo 20N_consolidated.txt:  31832529 bytes =  30.36 MB


Para cargar los archivos de blogs se hizo primero un preprocesamiento de los archivos XML puesto que hubo muchos errores a la hora de leerlos. La estrategia fue la siguiente:
* Se eliminaron los tags de formato del XML
* Se eliminaron las fechas de los posts
* Se dejó unicamente el contenido de los posts

Esto para poder leer los archivos como texto plano. Se aprovecha para saber qué palabras tienen frecuencia 1 por cada archivo para modelar el TKN \<UNK>

In [3]:
# Recorremos cada archivo de blogs
blogs_texts = []
for filename in os.listdir(BLOGS_FOLDER):
    extracted_text = ''
    with open(BLOGS_FOLDER + '/' + filename, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            # Agregamos solo las lineas del contenido del archivo
            if not line.startswith('From:') and not line.startswith('Subject:') and not line.startswith('Fax:') and not line.startswith('Phone:') and not line.startswith('Email:') and not line.startswith('INTERNET:'):
                extracted_text += line + " "
        blogs_texts.append(extracted_text + "<NEW_DOCUMENT>")

# Guardamos todos los textos en un unico .txt
with open('BAC_consolidated.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts:
        file.write(text)

# Revisamos el tamaño del archivo .txt
print('Tamaño del archivo BAC_consolidated.txt: ', os.path.getsize('BAC_consolidated.txt'), 'bytes = ', round(os.path.getsize('BAC_consolidated.txt') / 1048576, 2), 'MB')

Tamaño del archivo BAC_consolidated.txt:  757660374 bytes =  722.56 MB


## **II y III. Crear archivos de Training y Test y Tokenizar por sentencia**

Primero se crean los archivos de training y test

In [4]:
import re

# Cargamos los textos de los archivos .txt
with open('20N_consolidated.txt', 'r', encoding='utf-8', errors='ignore') as file:
    news_texts = file.read()

with open('BAC_consolidated.txt', 'r', encoding='utf-8', errors='ignore') as file:
    blogs_texts = file.read()

# Separar cada archivo en oraciones
news_texts = news_texts.split('.')
blogs_texts = blogs_texts.split('.')

# Quitar los textos vacios
news_texts = [text for text in news_texts if text != '']
blogs_texts = [text for text in blogs_texts if text != '']

print('Cantidad de oraciones en 20news-18828: ', len(news_texts))
print('Cantidad de oraciones en BAC: ', len(blogs_texts))

# Dividir los textos en 80% train y 20% test
news_texts_train = news_texts[:int(len(news_texts) * 0.8)]
news_texts_test = news_texts[int(len(news_texts) * 0.8):]
blogs_texts_train = blogs_texts[:int(len(blogs_texts) * 0.8)]
blogs_texts_test = blogs_texts[int(len(blogs_texts) * 0.8):]

# Guardar los textos de train y test en archivos .txt
with open('20N_GROUP_training.txt', 'w', encoding='utf-8') as file:
    for text in news_texts_train:
        file.write(text.strip() + '. ')

with open('20N_GROUP_testing.txt', 'w', encoding='utf-8') as file:
    for text in news_texts_test:
        file.write(text.strip() + '. ')

with open('BAC_GROUP_training.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts_train:
        file.write(text.strip() + '. ')

with open('BAC_GROUP_testing.txt', 'w', encoding='utf-8') as file:
    for text in blogs_texts_test:
        file.write(text.strip() + '. ')

# Revisamos el tamaño de los archivos .txt
print('Tamaño del archivo .txt de 20N_GROUP_training: ', os.path.getsize('20N_GROUP_training.txt'), 'bytes = ', round(os.path.getsize('20N_GROUP_training.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de 20N_GROUP_testing: ', os.path.getsize('20N_GROUP_testing.txt'), 'bytes = ', round(os.path.getsize('20N_GROUP_testing.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de BAC_GROUP_training: ', os.path.getsize('BAC_GROUP_training.txt'), 'bytes = ', round(os.path.getsize('BAC_GROUP_training.txt') / 1048576, 2), 'MB')
print('Tamaño del archivo .txt de BAC_GROUP_testing: ', os.path.getsize('BAC_GROUP_testing.txt'), 'bytes = ', round(os.path.getsize('BAC_GROUP_testing.txt') / 1048576, 2), 'MB')


Cantidad de oraciones en 20news-18828:  425274
Cantidad de oraciones en BAC:  9425954
Tamaño del archivo .txt de 20N_GROUP_training:  25475664 bytes =  24.3 MB
Tamaño del archivo .txt de 20N_GROUP_testing:  6323560 bytes =  6.03 MB
Tamaño del archivo .txt de BAC_GROUP_training:  599771948 bytes =  571.99 MB
Tamaño del archivo .txt de BAC_GROUP_testing:  148560451 bytes =  141.68 MB


In [5]:
print(blogs_texts_train[0])

      I've been selling books online since April, and am painfully aware of the vast ocean of book-knowledge that I haven't yet begun to cross


Procedemos a cargar los archivos de entrenamiento y a tokenizar

In [18]:
# Se cargan los archivos de training
with open('20N_GROUP_training.txt', 'r', encoding='utf-8', errors='ignore') as file:
    news_texts_train = file.read()

with open('BAC_GROUP_training.txt', 'r', encoding='utf-8', errors='ignore') as file:
    blogs_texts_train = file.read()

# Se separan los textos por "<NEW_DOCUMENT>" para obtener los documentos
news_texts_train = news_texts_train.split('<NEW_DOCUMENT>')
blogs_texts_train = blogs_texts_train.split('<NEW_DOCUMENT>')

print('Cantidad de documentos en 20N_GROUP_training: ', len(news_texts_train))
print('Cantidad de documentos en BAC_GROUP_training: ', len(blogs_texts_train))

# Se recorre cada documento y se encuentran los terminos con frecuencia 1 en cada documento
for text in news_texts_train:
    print(text)
    # Se normaliza el texto
    text = text.lower()
    # Se reemplazan todos los numeros por <NUM>
    text = re.sub(r'\d+', '<NUM>', text)
    # Se crea un diccionario donde guardamos la frecuencia de cada termino en el documento
    word_freq = {}
    # Se separa el texto en palabras
    words = text.split()
    print(words)
    # Se recorre cada palabra
    for word in words:
        if word != '<NUM>':
            # Si la palabra tiene un . al final, se quita
            if word.endswith('.'):
                word = word[:-1]
            # Se suma 1 a la cuenta de la palabra si existe
            if word in word_freq:
                word_freq[word] += 1
            # Se agrega la palabra al diccionario si no existe
            else:
                word_freq[word] = 1
    # Se recorre cada palabra del diccionario
    for word in word_freq:
        # Si la palabra tiene frecuencia 1, se reemplaza por <UNK>
        if word_freq[word] == 1:
            text = text.replace(' ' + word + ' ', ' <UNK> ')
    # Se reemplazan todos los . por </s> y <s>
    text = text.replace('.', ' </s> <s> ')
    # Se reemplazan todos los espacios en blanco por un solo espacio
    text = re.sub(r'\s+', ' ', text)
    # Se reemplazan los espacios en blanco al inicio y al final del texto
    text = text.strip()
    # Se agrega un <s> al inicio y un </s> al final del texto
    text = '<s> ' + text + ' </s>'
    print(text)
    break

            


Cantidad de documentos en 20N_GROUP_training:  14716
Cantidad de documentos en BAC_GROUP_training:  15324
The M code stream  might be independently attacked based on knowledge of clipper chip protocols as revealed plaintext. This could be invalidated by changing the temporal and or spatial relationship of the clipper M stream and the actual transmitted stream, under the control of a secure key generator synchronized between endpoints. The useful life time of captured law enforcement blocks might be limited based on hostile forces using them as targets following transmission interception. You would need a large number of them, but, hey there's supposed to be millions of these things, right?  Adding time stamps to the encrypted law enforcement block is probably impractical, who wants an encryption chip with a real time clock?  *****************************************************************************  The entire idea of the law enforcement block can be invalidated. I just had the thou

In [12]:
print(news_texts_train[0])

The M code stream  might be independently attacked based on knowledge of clipper chip protocols as revealed plaintext. This could be invalidated by changing the temporal and or spatial relationship of the clipper M stream and the actual transmitted stream, under the control of a secure key generator synchronized between endpoints. The useful life time of captured law enforcement blocks might be limited based on hostile forces using them as targets following transmission interception. You would need a large number of them, but, hey there's supposed to be millions of these things, right?  Adding time stamps to the encrypted law enforcement block is probably impractical, who wants an encryption chip with a real time clock?  *****************************************************************************  The entire idea of the law enforcement block can be invalidated. I just had the thought, that you could capture your own law enforcement blocks for session keys K that you will not use in ac

In [ ]:
# Se cargan los archivos de training
with open('20N_GROUP_training.txt', 'r', encoding='utf-8', errors='ignore') as file:
    news_texts_train = file.read()

with open('BAC_GROUP_training.txt', 'r', encoding='utf-8', errors='ignore') as file:
    blogs_texts_train = file.read()

# Normalizamos los textos pasando todo a minusculas
news_texts_train = news_texts_train.lower()
blogs_texts_train = blogs_texts_train.lower()

# Se separan los textos por "<NEW_DOCUMENT>" para obtener los documentos
news_texts_train = news_texts_train.split('<NEW_DOCUMENT>')
blogs_texts_train = blogs_texts_train.split('<NEW_DOCUMENT>')

# Para cada documento, se encuentran los términos con frecuencia 1
news_texts = ''
for text in news_texts_train:
    news_words = {}
    for word in text.split():
        if word in news_words:
            news_words[word] += 1
        else:
            news_words[word] = 1
    extracted_text = ''
    extracted_text += ' '.join([word if news_words[word] > 1 else '<UNK>' for word in text.split()])
    news_texts += extracted_text

blogs_texts = ''
for text in blogs_texts_train:
    blogs_words = {}
    for word in text.split():
        if word in blogs_words:
            blogs_words[word] += 1
        else:
            blogs_words[word] = 1
    extracted_text = ''
    extracted_text += ' '.join([word if blogs_words[word] > 1 else '<UNK>' for word in text.split()])
    blogs_texts += extracted_text

# Se reemplazan los números por el token <NUM>
news_texts= re.sub(r'\d+', '<NUM>', news_texts)
blogs_texts = re.sub(r'\d+', '<NUM>', blogs_texts)

# La marcación de TKN con frecuencia 1 se realizo en el paso anterior
# cuando se crearon los archivos consolidados

# Se separan los textos por '.' para obtener oraciones
news_texts_sentences = news_texts.split('.')
news_texts_sentences = blogs_texts.split('.')
print('Cantidad de oraciones en 20news-18828 training: ', len(news_texts))
print('Cantidad de oraciones en blogs training: ', len(blogs_texts))